In [1]:
import numpy as np

import json 
import requests
import urllib.request

from joblib import load

Загружаем тренировочные данные:

In [2]:
X_test_response = requests.get('https://drive.google.com/uc?id=16-H8e_5E7qMUlc2ekyfh0iEMbVbgXyFv')
with open('X_test.joblib', 'wb') as file:
    file.write(X_test_response.content)
X_test = load('X_test.joblib')

y_test_response = requests.get('https://drive.google.com/uc?id=1QC7uOgrNsEMsBz6suT4AMRFP8jWXKexF')
with open('y_test.joblib', 'wb') as file:
    file.write(y_test_response.content)
y_test = load('y_test.joblib')

Ункция для обработки данные и получения предсказания:

In [3]:
def send_json(x, url):
    x = tuple(x.values.tolist())
    FEATURE_NAMES = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 
                     'marital-status', 'occupation', 'relationship', 'race', 'sex',
                     'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
    body = {}
    for i, val in enumerate(x):
        try:
            body[FEATURE_NAMES[i]] = int(val)
        except ValueError:
            body[FEATURE_NAMES[i]] = val

    myurl = url + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    answer = response.json()['description'], response.json()['predictions']
    return answer

Посмотрим на предсказание для пяти случайных наблюдений из тренировочных данных, для каждого мы получим 2 строчки:

1) ```описание``` - профессия, пол, возраст, образование

2) ```предсказание``` - вероятность получать выше 50 тыс. долларов 

In [4]:
url = 'https://2266-35-234-58-134.ngrok-free.app/'

In [5]:
for i in range(6):
    response = send_json(X_test.iloc[np.random.randint(X_test.shape[0])], url=url)
    print(f'{response[0]}\n предсказание: {response[1]}\n')

 Farming-fishing,  Male, 42,  HS-grad
 предсказание: 0.27763511417154796

 Farming-fishing,  Male, 80,  7th-8th
 предсказание: 0.05919818728426948

 Adm-clerical,  Female, 23,  Some-college
 предсказание: 0.032463797678871954

 Adm-clerical,  Female, 56,  HS-grad
 предсказание: 0.06308698800627181

 Prof-specialty,  Male, 43,  Masters
 предсказание: 0.3044207094337827

 Exec-managerial,  Female, 27,  Bachelors
 предсказание: 0.03816101060208643



Предсказания получаем $\Rightarrow$ сервер работает. Посмотрим, сколько времени потребуется для 100 предсказаний:


In [6]:
%%time
N = 100
predictions = X_test.iloc[:N].apply(lambda x: send_json(x, url=url), axis=1)

CPU times: user 1.56 s, sys: 96.9 ms, total: 1.66 s
Wall time: 40.8 s


**Вывод**:

В этой работе мы:
- подготовили ML-модель классификации, определяющую вероятность получения заработной платы выше или ниже 50 тыс.долларов, исходя их переданных характеристик, таких как: пол,  возраст, образование, семейное положение, профессия и др. Качество модели на тренировочных данных:

    - F1_predict = 0.703603	
    - F1_best_thresh = 0.715368	
    - Precision = 0.682081	
    - Recall = 0.752071	
    - ROC AUC = 0.915359	
    - thresholds = 0.589355
		
- развернули Flask-сервер, реализующий работу модели
- проверили модель на работоспособность на отложенной тестовой выборке, получив 5 случайных предсказаний и замерив скорость получения 100 предсказаний. 